In [1]:
#creating Visualizations of Wage and Cost Data

In [2]:
import pandas as pd
import numpy as np

In [3]:
CostDF = pd.read_csv('StateCostDF.csv',index_col=0)
WageDF = pd.read_csv('StateWageDF.csv',index_col=0)
#salary and price parity
SalPar = pd.read_csv('combined.csv')

In [4]:
SalPar.rename(columns = {'Standardized Income':'Income v Parity'}, inplace = True)
SalPar.head()

,State,Year,Income,Price Parities,Income v Parity
0,Wyoming,2022,73248.0,91.903,79701.42
1,North Dakota,2022,70360.0,88.656,79362.93
2,Connecticut,2022,82938.0,106.412,77940.46
3,South Dakota,2022,68176.0,87.991,77480.65
4,Massachusetts,2022,84561.0,109.393,77300.19


In [5]:
CostSum = CostDF.where(CostDF['CostType'] != 'Required annual income after taxes').dropna(subset='CostType')
CostSum = CostSum.where(CostSum['CostType'] != 'Required annual income before taxes').dropna(subset='CostType')
CostSum = CostSum.groupby('State').sum()
CostSum.reset_index(inplace=True)
CostSum.head()

,State,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3
0,Alabama,32993.034563,61930.312125,76421.658617,99613.683170,51489.926625,67644.445887,85093.524195,97281.598837,51489.926625,69087.783448,87979.976141,101610.847355
1,Alaska,34161.516512,67520.447373,84789.881160,113688.950524,52803.378395,73038.275393,93222.616437,111400.193186,52803.378395,74515.447373,96177.312645,115832.013918
2,Arizona,36156.487428,66947.717365,81331.858797,108157.051030,55362.885828,72966.023725,90629.671967,106406.355926,55362.885828,74299.733393,93297.858797,110409.133533
3,Arkansas,32244.086011,62449.235073,78175.922171,102880.826867,50421.938317,67945.800593,86379.750664,99676.106610,50421.938317,69687.040851,89862.922171,104900.711328
4,California,45474.259096,91483.936928,113832.656346,153331.857250,69255.290699,95873.180990,119690.271296,145370.395042,69255.290699,99327.671669,126599.579909,155733.875518


In [6]:
WageTarg = WageDF.where(WageDF['WageType'] == 'Living Wage').dropna(subset='WageType')
WageTarg.drop(['WageType'],axis=1,inplace=True)
for column in WageTarg.columns:
    if column == 'State':
        continue
    else:
        WageTarg[column] = WageTarg[column].apply(lambda x: x * 40 * 52)


In [7]:
WageTarg.head()

,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State
0,33114.915788,62054.246717,76544.334810,99738.205569,51612.521248,61595.396007,72874.169363,78887.110521,25809.220043,34609.061681,44051.616332,50864.492500,Alabama
0,34280.480195,67642.886213,84913.794994,113811.103635,52929.011954,64050.334197,75122.628965,84186.281371,26463.929239,37318.793530,48153.351941,57974.872588,Alaska
0,36275.466320,67069.952979,81450.339267,108276.477583,55486.150046,67064.690618,78693.533731,88444.023741,27750.162560,37209.010324,46705.814669,55262.546193,Arizona
0,32368.403323,62570.102650,78297.907181,103005.832458,50541.943300,61034.253372,72444.370297,78705.079042,25272.616009,34903.886066,44994.361117,52513.252314,Arkansas
0,45592.972704,91606.263078,113954.894443,153453.802093,69375.277690,84592.612254,97006.873819,111285.562942,34690.781892,49727.283090,63361.176704,77928.540965,California


In [8]:
WagevCost = pd.DataFrame(columns=WageTarg.columns)
for column in WageTarg.columns:
    if column == 'State':
        continue
    else:
        Wage = list(WageTarg[column])
        Cost = list(CostSum[column])
        WvC = []
        if 'BOTH WORKING' in str(column):
            for i in range(len(Wage)):
                WvC.append(2*Wage[i] / Cost[i])
        else:
            for i in range(len(Wage)):
                WvC.append(Wage[i] / Cost[i])
        #print(len(WvC))
        WagevCost[column] = [x for x in WvC]
#
#print(adding)

In [9]:
WagevCost.drop(['2 ADULTS(1 WORKING) - 0','2 ADULTS(1 WORKING) - 1',
             '2 ADULTS(1 WORKING) - 2','2 ADULTS(1 WORKING) - 3'],axis=1,inplace=True)
WagevCost['State'] = [x for x in WageTarg['State']]
WagevCost.head()
#map this data in a visualization

,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State
0,1.003694,1.002001,1.001605,1.001250,1.002496,1.001887,1.001401,1.001163,Alabama
1,1.003482,1.001813,1.001461,1.001074,1.002357,1.001639,1.001345,1.001016,Alaska
2,1.003291,1.001826,1.001457,1.001104,1.002483,1.001592,1.001219,1.001050,Arizona
3,1.003856,1.001935,1.001560,1.001215,1.002445,1.001732,1.001400,1.001199,Arkansas
4,1.002611,1.001337,1.001074,1.000795,1.001823,1.001278,1.000970,1.000791,California


In [10]:
#average the wage and cost data horizontally
#add that to the SalPar dataframe
#map that out

#SalPar.set_index('State',inplace=True)
#adding.set_index('State',inplace=True)
#SalPar = SalPar.join(adding)
#SalPar = SalPar.dropna()
#SalPar